# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
import pandas as pd

In [ ]:
# your code here

df = pd.read_csv('tic-tac-toe.csv')
df.head()

In [ ]:
df.dtypes

In [ ]:
df = pd.get_dummies(df, columns=['TL','TM','TR','ML','MM','MR','BL','BM','BR'])

In [ ]:
def bool(row):
    if row == True:
        return 1
    else:
        return 0

df['class'] = df['class'].apply(bool)

In [ ]:
inputs = df.drop(columns = 'class')
outputs = df['class']

In [ ]:
inputs

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
# your code here

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, 
                                                    test_size = 0.2, random_state=0)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Activation, Dense

model = keras.Sequential()
model.add(Dense(27, input_dim = 27, activation = 'relu'))
model.add(Dense(18, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(2, activation = tf.nn.softmax))

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

model.fit(X_train, y_train, epochs=20)

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss, val_acc)

In [ ]:
model.save('tic-tac-toe.model')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here

new_model = tf.keras.models.load_model('tic-tac-toe.model')

In [ ]:
pred = model.predict(X_test[:10])
prediction = list(pred)

p = []
for i in range(0,10):
    a = prediction[i]
    b = a[0]
    c = a[1]
    if b>c:
        p.append(0)
    else:
        p.append(1)
        
display(p)
display(list(y_test[:10]))

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

'''
In fact, I've optimized my model before by changing various parameters. I didn't add more layers,
but I changed the number of units and epochs several times to get the best result. so in the end I have
very good accuracy: 0.9792 and loss rate: 0.0458. At the moment I'm fine with the model, but I definitely will
use these tips for future models
'''

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here